In [66]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math

In [67]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [68]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [69]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [70]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [71]:
emp_salary_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="emp_salary", properties=jdbc_properties))

In [72]:
emp_salary_df.show()

+------+------+------+-------+
|emp_id|  name|salary|dept_id|
+------+------+------+-------+
|   101| sohan|  3000|     11|
|   102| rohan|  4000|     12|
|   103| mohan|  5000|     13|
|   104|   cat|  3000|     11|
|   105|suresh|  4000|     12|
|   109|mahesh|  7000|     12|
|   108| kamal|  8000|     11|
+------+------+------+-------+



In [73]:
emp_salary_df1 = emp_salary_df.groupBy("salary", "dept_id").count().where("count > 1")
emp_salary_df1.show()

+------+-------+-----+
|salary|dept_id|count|
+------+-------+-----+
|  3000|     11|    2|
|  4000|     12|    2|
+------+-------+-----+



In [74]:

join_df = emp_salary_df1.join(emp_salary_df, (emp_salary_df1["dept_id"] == emp_salary_df["dept_id"]) \
                              & (emp_salary_df1["salary"] == emp_salary_df["salary"]),'inner')\
                             .drop(*[emp_salary_df["salary"], emp_salary_df["dept_id"], emp_salary_df1["count"]])
join_df.show()

+------+------+------+-------+
|emp_id|  name|salary|dept_id|
+------+------+------+-------+
|   101| sohan|  3000|     11|
|   104|   cat|  3000|     11|
|   102| rohan|  4000|     12|
|   105|suresh|  4000|     12|
+------+------+------+-------+

